In [1]:
import gradio 

In [2]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image
import requests
# Download human-readable labels for ImageNet.
response = requests.get("https://git.io/JJkYN")
labels = response.text.split("\n")

mobile_net = tf.keras.applications.MobileNetV2(weights="imagenet")
inception_net = tf.keras.applications.InceptionV3(weights="imagenet")

In [3]:
## labels

In [4]:
def classify_image_with_mobile_net(im):
    im = Image.fromarray(im.astype('uint8'), 'RGB')
    im = im.resize((224, 224))
    arr = np.array(im).reshape((-1, 224, 224, 3))
    arr = tf.keras.applications.mobilenet.preprocess_input(arr)
    prediction = mobile_net.predict(arr).flatten()
    return {labels[i]: float(prediction[i]) for i in range(1000)}

In [5]:
def classify_image_with_inception_net(im):
    try:
        im = Image.fromarray(im.astype("uint8"), "RGB")
        im = im.resize((299, 299))
        arr = np.array(im).reshape((-1, 299, 299, 3))
        arr = tf.keras.applications.inception_v3.preprocess_input(arr)
        prediction = inception_net.predict(arr).flatten()
        return {labels[i]: float(prediction[i]) for i in range(len(prediction))}
    except Exception as e:
        return {"error": str(e)}

In [6]:
# wrapper function that calls both models
def compare_models(img):
    return (
        classify_image_with_mobile_net(img),
        classify_image_with_inception_net(img)
    )

# Inputs and outputs
imagein = gr.Image(type="numpy")
label1 = gr.Label(num_top_classes=3, label="MobileNet")
label2 = gr.Label(num_top_classes=3, label="InceptionNet")

# Interface
demo = gr.Interface(
    fn=compare_models,              # <-- pass ONE function
    inputs=imagein,
    outputs=[label1, label2],       # two outputs
    examples=[
        ["cat.jpg"],
        ["sailboat.jpg"],
        ["bicycle.jpeg"],
    ],
    title="MobileNet vs. InceptionNet",
    description="""Let's compare 2 state-of-the-art machine learning models 
    that classify images into one of 1,000 categories."""
)

demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\manpr\anaconda3\lib\site-packages\gradio\queueing.py", line 745, in process_events
    response = await route_utils.call_process_api(
  File "C:\Users\manpr\anaconda3\lib\site-packages\gradio\route_utils.py", line 349, in call_process_api
    output = await app.get_blocks().process_api(
  File "C:\Users\manpr\anaconda3\lib\site-packages\gradio\blocks.py", line 2123, in process_api
    result = await self.call_function(
  File "C:\Users\manpr\anaconda3\lib\site-packages\gradio\blocks.py", line 1630, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "C:\Users\manpr\anaconda3\lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "C:\Users\manpr\anaconda3\lib\site-packages\anyio\_backends\_asyncio.py", line 2476, in run_sync_in_worker_thread
    return await future
  File "C:\Users\manpr\anaconda3\lib\site-packages\anyio\_ba